In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scripts.locator import Locator

### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [2]:
df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_airports = pd.read_csv("../data/processed/filtered_airports.csv")
df_railway = pd.read_csv("../data/processed/filtered_trains.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/filtered_municipalities.csv'

In [ ]:
df_municipalities.head()

In [ ]:
df_airports.head()

In [ ]:
df_railway.sample(15)

In [ ]:
# Apply the function to each row in the towns DataFrame and create two new columns
# df_municipalities[['closest_distance_airport', 'nearest_airport']] = Locator(df_municipalities, df_airports, 'airport_name').result


In [ ]:
df_municipalities.query("municipality_name_clean == 'amurrio'")

In [ ]:
# df_municipalities[['closest_distance_train', 'nearest_train_station']]  = Locator(df_municipalities, df_railway, 'station_name').result

In [ ]:
df_municipalities

In [ ]:
df_accessibility = df_municipalities[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance", "nearest_train_station", "closest_distance_train"]]

In [18]:
# df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
df_accessibility = pd.read_csv("../../data/scored/accessibility.csv")
df_hospitals = pd.read_csv("../../data/processed/filtered_hospitals.csv")

In [14]:
df_accessibility

,province,municipality_name,population,male,female,municipality_name_clean,province_clean,altitude,longitude,latitude,cmun,closest_distance_airport,nearest_airport,closest_distance_train,nearest_train_station
0,Álava,Alegría-Dulantzi,2971,1531,1440,alegria-dulantzi,alava,561.68570,-2.513507,42.84149,1001,18.273217,Aeropuerto de Vitoria,0.819736,Alegria-Dulantzi
1,Álava,Amurrio,10330,5149,5181,amurrio,alava,219.69100,-3.001022,43.05265,1002,28.762394,Aeropuerto de Bilbao,30.756835,Nanclares-Langraiz
2,Álava,Aramaio,1381,709,672,aramaio,alava,381.87970,-2.566000,43.05400,1003,23.639652,Aeropuerto de Vitoria,23.176277,Alegria-Dulantzi
3,Álava,Artziniega,1856,913,943,artziniega,alava,196.98080,-3.128209,43.12220,1004,26.840606,Aeropuerto de Bilbao,43.021998,Nanclares-Langraiz
4,Álava,Armiñón,247,127,120,arminon,alava,463.58150,-2.872574,42.72305,1006,20.846277,Aeropuerto de Vitoria,2.240750,Manzanos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7947,Zaragoza,Biel,170,104,66,biel,zaragoza,754.24450,-0.936588,42.38749,50901,61.667966,Aeropuerto de Huesca Pirineos,17.531651,Riglos
7948,Zaragoza,Marracos,85,43,42,marracos,zaragoza,404.73610,-0.776047,42.09059,50902,37.961861,Aeropuerto de Huesca Pirineos,12.495085,Valença Do Minho
7949,Zaragoza,Villamayor de Gállego,2854,1446,1408,villamayor de gallego,zaragoza,231.12340,-0.773315,41.68518,50903,23.955690,Aeropuerto de Zaragoza,9.820085,Villanueva De Gallego
7950,Ceuta,Ceuta,83229,41980,41249,ceuta,ceuta,13.47725,-5.316195,35.88829,51001,1.008709,Helipuerto de Ceuta,29.010931,Algeciras


In [16]:
df_accessibility = df_accessibility[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]

In [17]:
df_accessibility

,cmun,municipality_name,province,altitude,longitude,latitude,nearest_airport,closest_distance_airport,nearest_train_station,closest_distance_train
0,1001,Alegría-Dulantzi,Álava,561.68570,-2.513507,42.84149,Aeropuerto de Vitoria,18.273217,Alegria-Dulantzi,0.819736
1,1002,Amurrio,Álava,219.69100,-3.001022,43.05265,Aeropuerto de Bilbao,28.762394,Nanclares-Langraiz,30.756835
2,1003,Aramaio,Álava,381.87970,-2.566000,43.05400,Aeropuerto de Vitoria,23.639652,Alegria-Dulantzi,23.176277
3,1004,Artziniega,Álava,196.98080,-3.128209,43.12220,Aeropuerto de Bilbao,26.840606,Nanclares-Langraiz,43.021998
4,1006,Armiñón,Álava,463.58150,-2.872574,42.72305,Aeropuerto de Vitoria,20.846277,Manzanos,2.240750
...,...,...,...,...,...,...,...,...,...,...
7947,50901,Biel,Zaragoza,754.24450,-0.936588,42.38749,Aeropuerto de Huesca Pirineos,61.667966,Riglos,17.531651
7948,50902,Marracos,Zaragoza,404.73610,-0.776047,42.09059,Aeropuerto de Huesca Pirineos,37.961861,Valença Do Minho,12.495085
7949,50903,Villamayor de Gállego,Zaragoza,231.12340,-0.773315,41.68518,Aeropuerto de Zaragoza,23.955690,Villanueva De Gallego,9.820085
7950,51001,Ceuta,Ceuta,13.47725,-5.316195,35.88829,Helipuerto de Ceuta,1.008709,Algeciras,29.010931


In [22]:
df_hospitals_general = df_hospitals.query("type == 'General'")

In [ ]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_accessibility[['closest_distance_hospital', 'nearest_hospital']] = Locator(df_accessibility, df_hospitals_general, 'hospital_name').result

In [ ]:
df_municipalities.to_csv("../data/scored/accessibility.csv", index=False)